In [ ]:
from copy import deepcopy
import time
import math
import numpy
from sys import exit

##Colors of pieces
redColor = "\u001b[31m"
resetColor = "\u001b[0m"
king_player_Color = "\u001B[34m"

num = 8  ## Size of checkers board
search_depth = 4    
num_of_evaluated_leaves = 0
num_of_branches = 0
num_of_pruned_min = 0
num_of_All_branches_min=0
num_of_pruned_max=0
num_of_All_branches_max=0

numofpieces = 0


## Class Node
class Node:
    def __init__(self, board, move=None, parent=None, value=None ):
        self.board = board
        self.value = value
        self.move = move
        self.parent = parent

    ##Get children nodes of the node
    def get_children(self, maximizing_player):
        current_state = deepcopy(self.board)
        possible_moves = [] 
        children_states = []
        big_letter = ""
        queen_row = 0
        ##If maximizing player get possible moves of computer
        if maximizing_player is True:
            possible_moves = Checkers.get_possible_moves_Computer(current_state)
            big_letter = "C"
            queen_row = num-1
        else:
            ##If minimizing player get possible moves of player
            possible_moves = Checkers.get_possible_moves_player(current_state)
            big_letter = "P"
            queen_row = 0
        ##Insert the nodes with certain moves in children states of the node
        for i in range(len(possible_moves)):
            old_i = possible_moves[i][0]
            old_j = possible_moves[i][1]
            new_i = possible_moves[i][2]
            new_j = possible_moves[i][3]
            state = deepcopy(current_state)
            Checkers.make_a_move(state, old_i, old_j, new_i, new_j, big_letter, queen_row)
            children_states.append(Node(state, [old_i, old_j, new_i, new_j]))
        return children_states

    def set_value(self, value):
        self.value = value

    def get_value(self):
        return self.value

    def get_board(self):
        return self.board

    def get_parent(self):
        return self.parent

    def set_parent(self, parent):
        self.parent = parent


class Checkers:
    def __init__(self):
        self.matrix =[]
        ##Make the matrix of board
        for i in range (num):
            self.matrix.append([])

        self.turn = True
        self.computer_pieces = numofpieces
        self.player_pieces = numofpieces
        self.possible_moves = []

        for row in self.matrix:
            for i in range(num):
                row.append("---")
        self.position_computer()
        self.position_player()

    ##Filling matrix board with pieces
    def position_computer(self):
        r2 = int((num-2)/2)
        for i in range(r2):
            for j in range(num):
                if (i + j) % 2 == 1:
                    self.matrix[i][j] = ("c" + str(i) + str(j))

    def position_player(self):
        r1 = int(num - ((num-2)/2))
        for i in range(r1, num, 1):
            for j in range(num):
                if (i + j) % 2 == 1:
                    self.matrix[i][j] = ("p" + str(i) + str(j))

    # Take the input coordinates from the player
    def get_player_input(self):
        global num
        
        #Get possible moves of the player
        possible_moves = Checkers.get_possible_moves_player(self.matrix)
        
        ##If there is no possible moves for player
        if len(possible_moves) == 0:
            if self.computer_pieces > self.player_pieces:
                print("You have no moves left, and you have fewer pieces than the computer.you lose")
                exit()
            else:
                print("You have no possible moves.\nGame ended")
                exit()
        self.player_pieces = 0
        self.computer_pieces = 0

        while True:
            # If user type enter input box the game will end
            which_Piece = input("Which piece[i,j]: ")
            if which_Piece == "":
                print("Game ended")
                exit()
                
            # If user type s input box so the player surrender and the game will end
            if which_Piece == 's':
                print("You surrender. Game ended")
                exit()

            # If user type enter inside input box the game will end
            Where_to = input("Where to[i,j]:")
            if Where_to == "":
                print("Game ended")
                exit()

            # If user type s input box so the player surrender and the game will end
            if Where_to == 's':
                print("You surrender. Game ended")
                exit()

            # get the old and new cooardinated from input
            old = which_Piece.split(",")
            new = Where_to.split(",")

            #Check if input is valid or not
            if len(old) != 2 or len(new) != 2:
                print("Ilegal input")
            else:
                old_i = old[0]
                old_j = old[1]
                new_i = new[0]
                new_j = new[1]
                if not old_i.isdigit() or not old_j.isdigit() or not new_i.isdigit() or not new_j.isdigit():
                    print("Ilegal input")
                else:
                    move = [int(old_i), int(old_j), int(new_i), int(new_j)]
                    ##Check if move can be done or not
                    if move not in possible_moves:
                        print("Ilegal move")
                    else:
                        # If input is valid so make the move of player
                        Checkers.make_a_move(self.matrix, int(old_i), int(old_j), int(new_i), int(new_j), "P", 0)
                        for m in range(num):
                            for n in range(num):
                                if self.matrix[m][n][0] == "c" or self.matrix[m][n][0] == "C":
                                    self.computer_pieces += 1
                                elif self.matrix[m][n][0] == "p" or self.matrix[m][n][0] == "P":
                                    self.player_pieces += 1
                        break


    ## Check if the computer's move can be done
    def check_moves(board, old_i, old_j, new_i, new_j):

        #If the coordinates not inside the board 
        if new_i > num-1 or new_i < 0:
            return False
        if new_j > num-1 or new_j < 0:
            return False
        #If the old coordinates have no piece and empty
        if board[old_i][old_j] == "---":
            return False
        #If the new coordinates input have piece and not empty
        if board[new_i][new_j] != "---":
            return False
        if board[old_i][old_j][0] == "p" or board[old_i][old_j][0] == "P":
            return False
        if board[new_i][new_j] == "---":
            return True

    ## Check if the jump can be done
    def check_jumps(board, old_i, old_j, via_i, via_j, new_i, new_j):
        global num
        #If the input coordinates not inside the board 
        if new_i > num-1 or new_i < 0:
            return False
        if new_j > num-1 or new_j < 0:
            return False
        if board[via_i][via_j] == "---":
            return False
        if board[via_i][via_j][0] == "C" or board[via_i][via_j][0] == "c":
            return False
        #If the where to input have piece and not empty
        if board[new_i][new_j] != "---":
            return False
        if board[old_i][old_j] == "---":
            return False
        if board[old_i][old_j][0] == "p" or board[old_i][old_j][0] == "P":
            return False
        return True


    ## moving Order
    #Get possible moves of computer
    def get_possible_moves_Computer(board):
        global num
        possible_moves = []
        possible_jumps = []
        for m in range(num):
            for n in range(num):
                #Get forward moves
                if board[m][n][0] == "c":
                    if Checkers.check_moves(board, m, n, m + 1, n - 1):
                        possible_moves.append([m, n, m + 1, n - 1])

                    if Checkers.check_moves(board, m, n, m + 1, n + 1):
                        possible_moves.append([m, n, m + 1, n + 1])

                    if Checkers.check_jumps(board, m, n, m + 1, n + 1, m + 2, n + 2):
                        possible_jumps.append([m, n, m + 2, n + 2]) 

                    if Checkers.check_jumps(board, m, n, m + 1, n - 1, m + 2, n - 2):
                        possible_jumps.append([m, n, m + 2, n - 2])

                #Get forward and backward moves
                elif board[m][n][0] == "C":
                    if Checkers.check_moves(board, m, n, m + 1, n + 1):
                        possible_moves.append([m, n, m + 1, n + 1])

                    if Checkers.check_moves(board, m, n, m + 1, n - 1):
                        possible_moves.append([m, n, m + 1, n - 1])

                    if Checkers.check_moves(board, m, n, m - 1, n - 1):
                        possible_moves.append([m, n, m - 1, n - 1])

                    if Checkers.check_moves(board, m, n, m - 1, n + 1):
                        possible_moves.append([m, n, m - 1, n + 1])

                    if Checkers.check_jumps(board, m, n, m + 1, n - 1, m + 2, n - 2):
                        possible_jumps.append([m, n, m + 2, n - 2])

                    if Checkers.check_jumps(board, m, n, m - 1, n - 1, m - 2, n - 2):
                        possible_jumps.append([m, n, m - 2, n - 2])

                    if Checkers.check_jumps(board, m, n, m - 1, n + 1, m - 2, n + 2):
                        possible_jumps.append([m, n, m - 2, n + 2])

                    if Checkers.check_jumps(board, m, n, m + 1, n + 1, m + 2, n + 2):
                        possible_jumps.append([m, n, m + 2, n + 2])

        if len(possible_jumps) == 0:
          return possible_moves
        else:
          return possible_jumps

    # Evaluate positions of computer pieces
    def evaluate_Positions(board):
        global num
        evaluation = 0
        computer_piece = 0
        player_piece = 0
        for i in range(num):
            for j in range(num):
                # Increment no of pieces of computer
                if board[i][j][0] == "c" or board[i][j][0] == "C":
                    computer_piece = computer_piece+ 1

                    #3 normal piece = 5
                    if board[i][j][0] == "c":
                        evaluation = evaluation+5

                    ## King position = 10
                    if board[i][j][0] == "C":
                        evaluation = evaluation + 10

                    ## in checkers, pieces at corners and sideways positions can’t be eaten. So we can create an evaluation function 
                    ## that gives higher values to pieces that lie on those board positions 
                    ## thus giving higher outcomes for game moves that place pieces in those positions.
                    if i == 0 or j == 0 or i == num-1 or j == num-1:
                        evaluation = evaluation + 7

                    if i + 1 > num-1 or j - 1 < 0 or i - 1 < 0 or j + 1 > num-1:
                        continue

                    ## in these cases the computer peice can be eaten by player
                    if (board[i + 1][j - 1][0] == "p" or board[i + 1][j - 1][0] == "P") and board[i - 1][j + 1] == "---":
                        evaluation = evaluation -3
                    if (board[i + 1][j + 1][0] == "p" or board[i + 1][j + 1] == "P") and board[i - 1][j - 1] == "---":
                        evaluation = evaluation -3
                    if board[i - 1][j - 1][0] == "P" and board[i + 1][j + 1] == "---":
                        evaluation = evaluation -3
                    if board[i - 1][j + 1][0] == "P" and board[i + 1][j - 1] == "---":
                        evaluation = evaluation -3

                    if i + 2 > num-1 or i - 2 < 0:
                        continue

                    #In these cases the computer piece can eat player piece
                    if (board[i + 1][j - 1][0] == "P" or board[i + 1][j - 1][0] == "p") and board[i + 2][j - 2] == "---":
                        evaluation = evaluation + 6
                    if i + 2 > num-1 or j + 2 > num-1:
                        continue
                    if (board[i + 1][j + 1][0] == "P" or board[i + 1][j + 1][0] == "p") and board[i + 2][j + 2] == "---":
                        evaluation = evaluation + 6

                # Increment no of pieces of player
                elif board[i][j][0] == "P" or board[i][j][0] == "P":
                    player_piece = player_piece+ 1
                    
        Final_evaluation = evaluation + (computer_piece - player_piece) * 1000
        return Final_evaluation

    ##Get possible moves of player
    def get_possible_moves_player(board):
        global num
        possible_moves = []
        possible_jumps = []

        for m in range(num):
            for n in range(num):
                #Get forward moves
                if board[m][n][0] == "p":
                    if Checkers.check_player_moves(board, m, n, m - 1, n - 1):
                        possible_moves.append([m, n, m - 1, n - 1])

                    if Checkers.check_player_moves(board, m, n, m - 1, n + 1):
                        possible_moves.append([m, n, m - 1, n + 1])

                    if Checkers.check_player_jumps(board, m, n, m - 1, n - 1, m - 2, n - 2):
                        possible_jumps.append([m, n, m - 2, n - 2])

                    if Checkers.check_player_jumps(board, m, n, m - 1, n + 1, m - 2, n + 2):
                        possible_jumps.append([m, n, m - 2, n + 2])

                #Get forward and backward moves
                elif board[m][n][0] == "P":
                    if Checkers.check_player_moves(board, m, n, m - 1, n - 1):
                        possible_moves.append([m, n, m - 1, n - 1])

                    if Checkers.check_player_moves(board, m, n, m - 1, n + 1):
                        possible_moves.append([m, n, m - 1, n + 1])

                    if Checkers.check_player_jumps(board, m, n, m - 1, n - 1, m - 2, n - 2):
                        possible_jumps.append([m, n, m - 2, n - 2])

                    if Checkers.check_player_jumps(board, m, n, m - 1, n + 1, m - 2, n + 2):
                        possible_jumps.append([m, n, m - 2, n + 2])

                    if Checkers.check_player_moves(board, m, n, m + 1, n - 1):
                        possible_moves.append([m, n, m + 1, n - 1])

                    if Checkers.check_player_jumps(board, m, n, m + 1, n - 1, m + 2, n - 2):
                        possible_jumps.append([m, n, m + 2, n - 2])

                    if Checkers.check_player_moves(board, m, n, m + 1, n + 1):
                        possible_moves.append([m, n, m + 1, n + 1])

                    if Checkers.check_player_jumps(board, m, n, m + 1, n + 1, m + 2, n + 2):
                        possible_jumps.append([m, n, m + 2, n + 2])

        if len(possible_jumps) == 0:
          return possible_moves
        else:
          return possible_jumps

    ## Check if player's move is valide to be done
    def check_player_moves(board, old_i, old_j, new_i, new_j):
        global num
        #If the input coordinates not inside the board 
        if new_i > num-1 or new_i < 0:
            return False
        if new_j > num-1 or new_j < 0:
            return False
        if board[old_i][old_j] == "---":
            return False
         #If the where to input have piece and not empty
        if board[new_i][new_j] != "---":
            return False
        if board[old_i][old_j][0] == "c" or board[old_i][old_j][0] == "C":
            return False
        if board[new_i][new_j] == "---":
            return True

    ## Check if player's jump is valide to be done
    def check_player_jumps(board, old_i, old_j, via_i, via_j, new_i, new_j):
        #If the input coordinates not inside the board 
        if new_i > num-1 or new_i < 0:
            return False
        if new_j > num-1 or new_j < 0:
            return False
        if board[via_i][via_j] == "---":
            return False
        if board[via_i][via_j][0] == "P" or board[via_i][via_j][0] == "p":
            return False
        #If the where to input have piece and not empty
        if board[new_i][new_j] != "---":
            return False
        if board[old_i][old_j] == "---":
            return False
        if board[old_i][old_j][0] == "c" or board[old_i][old_j][0] == "C":
            return False
        return True

    ## Evaluate the states and get the value of each state by using alpha beta pruning algorithm
    def evaluate_states(self):
        global num_of_branches
        global num_of_evaluated_leaves
        num_of_evaluated_leaves =0
        current_state = Node(deepcopy(self.matrix))

        ##Get the children nodes of current state
        first_computer_moves = current_state.get_children(True)
        
        ## Chec if the curent state has children nodes or not
        if len(first_computer_moves) == 0:
            if self.player_pieces > self.computer_pieces:
                print("Computer has no possible moves, you have more pieces.\nYou are the winner")
                exit()
            else:
                print("Computer has no possible moves.\nEnd Game")
                exit()
        dict = {}


        ## Improving move ordering 
        evaluation_arr = []
        for i in range(len(first_computer_moves)):
            child = first_computer_moves[i]
            evaluation_value = Checkers.evaluate_Positions(child.get_board())
            evaluation_arr.append(evaluation_value)

        indices_arr = Checkers.Reverse(numpy.argsort(evaluation_arr))
        first_computer_moves_Improved=[]
        for e in indices_arr:
            first_computer_moves_Improved.append(first_computer_moves[e])


        num_of_branches =0
        ##Loop on the children nodes of current state to get the value of current state 
        for i in range(len(first_computer_moves)):
            child = first_computer_moves[i]
            value = Checkers.minimax(child.get_board(), search_depth, -math.inf, math.inf, False)
            dict[value] = child

        print ("Num of evaluated leaves are " +str(num_of_evaluated_leaves))

        if len(dict.keys()) == 0:
            print("Computer has cornered itself.\nYou are the winner")
            exit()
        new_board = dict[max(dict)].get_board()
        move = dict[max(dict)].move
        self.matrix = new_board
        print("Computer moved (" + str(move[0]) + "," + str(move[1]) + ") to (" + str(move[2]) + "," + str(
            move[3]) + ").")


    ## Apply alpha beta pruning algorithm
    def minimax(board, depth, alpha, beta, maximizing_player):

        global num_of_evaluated_leaves
        global num_of_pruned_min
        global num_of_All_branches_min
        global num_of_pruned_max
        global num_of_All_branches_max

        ## if depth =0 so get the value of node
        if depth == 0:
            num_of_evaluated_leaves = num_of_evaluated_leaves + 1
            return Checkers.evaluate_Positions(board)
        
        current_state = Node(deepcopy(board))
        if maximizing_player is True:
            max_eval = -math.inf
            for child in current_state.get_children(True):
                ev = Checkers.minimax(child.get_board(), depth - 1, alpha, beta, False)
                max_eval = max(max_eval, ev)
                alpha = max(alpha, ev)
                num_of_All_branches_max = num_of_All_branches_max+1
                #Cut off
                if beta <= alpha:
                    num_of_pruned_max= num_of_pruned_max + 1
                    break
            current_state.set_value(max_eval)
            return max_eval
        else:
            min_eval = math.inf
            for child in current_state.get_children(False):
                ev = Checkers.minimax(child.get_board(), depth - 1, alpha, beta, True)
                min_eval = min(min_eval, ev)
                beta = min(beta, ev)
                num_of_All_branches_min = num_of_All_branches_min+1
                #Cut off
                if beta <= alpha:
                    num_of_pruned_min= num_of_pruned_min + 1
                    break
            current_state.set_value(min_eval)
            return min_eval


    ##Make the move by updating the board matrix
    def make_a_move(board, old_i, old_j, new_i, new_j, big_letter, queen_row):
        letter = board[old_i][old_j][0]
        i_difference = old_i - new_i
        j_difference = old_j - new_j
        
        if i_difference == -2 and j_difference == 2:
            board[old_i + 1][old_j - 1] = "---"

        elif i_difference == 2 and j_difference == 2:
            board[old_i - 1][old_j - 1] = "---"

        elif i_difference == 2 and j_difference == -2:
            board[old_i - 1][old_j + 1] = "---"

        elif i_difference == -2 and j_difference == -2:
            board[old_i + 1][old_j + 1] = "---"
        ##Check if the move will result that piece become king
        if new_i == queen_row:
            letter = big_letter
        
        ##Put in the place of old coordinates -----
        board[old_i][old_j] = "---"
        ##Put in the place of new coordinates the piece letter beside y and x coordinate
        board[new_i][new_j] = letter + str(new_i) + str(new_j)

    def Reverse(lst): 
        return [ele for ele in reversed(lst)] 

    ## Drawing board function
    def draw_Board(self):
        i = 0
        print()
        for row in self.matrix:
            print(i, end="  |")
            i = i + 1
            for elem in row:
                ##give red color for computer pieces
                if elem[0] =='c' or elem[0] =='C':
                    print(redColor + elem + resetColor, end=" ")
                
                ##give blue color for king player pieces
                elif  elem[0] =='P':
                    print(king_player_Color + elem + resetColor, end=" " )
                else:
                    print(elem , end=" ")
            print("\n")

        for j in range(num):
            if j == 0:
                j = "     0"
            print(j, end="   ")
        print("\n")     
    def play(self):
        print("\nRules of Game:")
        print("1.Enter the coordinates in the form i,j.")
        print("2.Exit the game by pressing enter.")
        print("3.Surrender pressing 's'.")

        global num
        global numofpieces 
        ##No. of pieces of board
        numofpieces = (num/2)* (num-2)/2
        
        
        while True:
            
            #Draw board
            self.draw_Board()
            #Get input coordinates of player
            if self.turn is True:
                print("\nPlayer's turn.")
                self.get_player_input()
            else:
                print("Computer's turn.")
                global num_of_evaluated_states
                global num_of_pruned
                num_of_pruned=0
                num_of_evaluated_leaves =0
                print("With search depth = "+str(search_depth))
                t1 = time.time()
                ##Apply evaluate states function
                self.evaluate_states()
                t2 = time.time()
                diff = t2 - t1
                
                ##Prin the statistics
                print("Run time of evaluating states is  " + str(diff) + " seconds.")
                print ("Number of all Branches  in the max value " + str (num_of_All_branches_max))
                print ("Number of pruned branches in the max value " + str(num_of_pruned_max))
                print ("Number of unpruned branches in the max value " + str(num_of_All_branches_max - num_of_pruned_max))

                print ("Number of all Branches  in the min value " + str (num_of_All_branches_min))
                print ("Number of pruned branches in the min value " + str(num_of_pruned_min))
                print ("Number of unpruned branches in the min value " + str(num_of_All_branches_min - num_of_pruned_min))


            ##If the number player's pieces =0 so he is loser
            if self.player_pieces == 0:
                self.draw_Board()
                print(redColor + "You have no pieces.\nYou lose" + resetColor)
                exit()
            
            ##If the number computer's pieces =0 so player is winner
            elif self.computer_pieces == 0:
                self.draw_Board()
                print("Computer has no pieces.\nYou win")
                exit()
            elif self.computer_pieces - self.player_pieces == num-1:
                decision = input("You have 7 pieces smaller than  computer.Do you want to surrender?")
                if decision == "" or decision == "yes":
                    print("Coward.")
                    exit()
            self.turn = not self.turn

if __name__ == '__main__':
    global num
    ##Take the size of board 8,10,etc
    n = input("Enter N which is size of board")
    num = int (n)
    checkers = Checkers()
    checkers.play()




Enter N which is size of board8

Rules of Game:
1.Enter the coordinates in the form i,j.
2.Exit the game by pressing enter.
3.Surrender pressing 's'.

0  |--- c01 --- c03 --- c05 --- c07 

1  |c10 --- c12 --- c14 --- c16 --- 

2  |--- c21 --- c23 --- c25 --- c27 

3  |--- --- --- --- --- --- --- --- 

4  |--- --- --- --- --- --- --- --- 

5  |p50 --- p52 --- p54 --- p56 --- 

6  |--- p61 --- p63 --- p65 --- p67 

7  |p70 --- p72 --- p74 --- p76 --- 

     0   1   2   3   4   5   6   7   


Player's turn.
Which piece[i,j]: 5,4
Where to[i,j]:4,3

0  |--- c01 --- c03 --- c05 --- c07 

1  |c10 --- c12 --- c14 --- c16 --- 

2  |--- c21 --- c23 --- c25 --- c27 

3  |--- --- --- --- --- --- --- --- 

4  |--- --- --- p43 --- --- --- --- 

5  |p50 --- p52 --- --- --- p56 --- 

6  |--- p61 --- p63 --- p65 --- p67 

7  |p70 --- p72 --- p74 --- p76 --- 

     0   1   2   3   4   5   6   7   

Computer's turn.
With search depth = 4
Num of evaluated leaves are 332
Computer moved (2,3) to (3,4).
Run 